## What is selenium?

The outline for this tutorial was taken from <a href = 'http://zetcode.com/python/selenium/'> zetcode's selenium tutorial</a>

Selenium is a tool to test your web application

- permit it to tap on buttons
- enter content in structures
- skim your site to check withether everything is "OK" 

In [1]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


## Selenium unittest example

`unittest` is a python unit testing framework Unittest supports test automation, sharing of setup and shutdown code for tests, aggregation of tests into collections, and independence of the tests from the reporting framework. 

Unittest provides a base class `TestCase` which can be used to create new test cases.

The `setUp()` is a hook method for setting up the test fixture before exercising it and the `tearDown()` is a hook method for deconstructing the rtest fixture after testing it. 

A **test ficture** represents the preparation needed to perform one or more tests, for example creating temporary or proxy databases, directories, or starting a server process. A **test case** is the test function which refers to the individual test. 

**The name of the function must begin with `test`.** 

The checks are performed with various assertion methods, such as `assertIn()`, `assertTrue()`, `assertEqual()`.

In [38]:
%%writefile unittest_example.py
import unittest

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

class WebCode(unittest.TestCase):
    
    def setUp(self):
        opts = Options()
        opts.headless = True
        
        self.driver = webdriver.Firefox(options=opts)
    
    def test_title(self):
        
        self.driver.get('http://webcode.me')
        self.assertIn("My html page", self.driver.title)
        
        
    def test_paragraphs(self):
        
        self.driver.get('http://webcode.me')
        
        els = self.driver.find_elements_by_tag_name('p')
        
        self.assertIn('Today is a beautiful day', els[0].text)
        self.assertIn('Hello there', els[1].text)
        
    def tearDown(self):
        self.driver.close()
        
if __name__ == "__main__":
    unittest.main()

Overwriting unittest_example.py


In [39]:
! python3 unittest_example.py

..
----------------------------------------------------------------------
Ran 2 tests in 14.520s

OK


## Selenium with PyTest

The `pytest` module is a python library for testing Python applications.

The pytest looks for `test_*.py` or `*_test.py` files in directories. 

In [40]:
! pip install pytest

In [41]:
! mkdir tests

In [54]:
%%writefile Selenium/tests//test_web.py
#!/usr/bin/env python3

import pytest

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options


@pytest.fixture
def browser():

    opts = Options()
    opts.headless = True
    driver = Firefox(options=opts)

    driver.implicitly_wait(5)

    yield driver

    # For cleanup, quit the driver
    driver.quit()


def test_get_title(browser):
    browser.get("http://webcode.me")

    assert 'My html page' == browser.title

Overwriting Selenium/tests//test_web.py


In [55]:
!python3 Selenium/tests/test_web.py

## Python Selenium Flask Example

In the next example, we create a test case with pytest and Selenium for a Flask web application. We test for a response from an HTML form.

    app.py
    ├───static
    │       greet.html
    ├───templates
    │       index.html
    └───tests
            web_test.py
            
#### 1- `greet.html` we have a greet form in the static resources, ther form sends a text value to the flask app

#### 2- `index.html` This is the Flask template file, which returns a message back to the client

#### 3- `app.py` The Flask app has 2 routes: one for the homepage and one for the greeting

#### 4- `web_test.py` contains a test case for the greet form


In [56]:
!mkdir Selenium/static 
!mkdir Selenium/templates

mkdir: Selenium/tests: File exists


In [57]:
%%writefile Selenium/static/greet.html

<!doctype html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Greet form</title>
</head>

<body>

    <p>Enter your name:</p>

    <form id="myform" action="greet">

        <input name="name" type="text">
        <button type="submit">Submit</button>

    </form>

</body>

</html>


Writing Selenium/static/greet.html


In [58]:
%%writefile Selenium/templates/index.html

<!doctype html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <title>Greeting</title>
</head>

<body>
    <p>
        Hello {{ name }}
    </p>
</body>

</html>

Writing Selenium/templates/index.html


In [59]:
%%writefile Selenium/app.py

#!/usr/bin/env python3

from flask import Flask, render_template, request

app = Flask(__name__)


@app.route("/")
def home():
    return app.send_static_file('greet.html')


@app.route("/greet")
def greet():
    username = request.args.get('name')
    return render_template('index.html', name=username)


if __name__ == "__main__":
    app.run()

Writing Selenium/app.py


In [60]:
%%writefile Selenium/tests/web_test.py
#!/usr/bin/env python3

import pytest

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait


@pytest.fixture
def browser():
    opts = Options()
    opts.headless = True
    driver = Firefox(options=opts)

    driver.implicitly_wait(10)

    yield driver

    # For cleanup, quit the driver
    driver.quit()


def test_greet_form(browser):
    user_name = "John"

    browser.get('http://localhost:5000/')

    form = browser.find_element_by_id("myform")
    name = browser.find_element_by_name("name")
    name.send_keys(user_name)

    form.submit()

    WebDriverWait(browser, 12).until(ec.url_matches('/greet'),
                                     'Timed out waiting for response')

    content = browser.page_source
    print(content)
    assert 'Hello John' in content

Writing Selenium/tests/web_test.py


In [61]:
!set FLASK_APP=app.py

In [62]:
!flask run

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


### Now, in another terminal, run `pytest` to see the results of the test